# UFO 1 data analysis part 1

Big Data házi feladat, UFO1 feladatkör EDA analízise, adatfelderítése.

A használathoz feltételezzük, hogy már létezik egy állomány, data.csv néven, ami tartalmazza a szükséges adatokat.

Ennek a notebooknak a célja, hogy a letöltött adatokkal megismerkedjünk, néhány megfigyelést állapítsunk meg róluk, esetlegesen a nem megfelelő adatokat kiemeljük, és töröljük a készletből.


## Adatok betöltése, átalakítása

Az egyszerű betöltés után néhány kisebb módosítást végeztünk az adatokon:
* Date / Time osztlop átnevezése Date-re, az kezelhetőbbség kedvéért
* Shape oszlop értékei nem voltak konzisztensen kis vagy nagybetűsek, ezért ezeket egységesen csupa lowercase-re konvertáltuk
* A dátumok feldolgozásához a Python saját parser-ét használtuk
    * Amit az nem volt képes feldolgozni, azt egy nem előforduló értékre állítottuk, amit majd később ki lehet szűrni / el lehet dobni
    * Mivel az eredeti adatok szintaxisa miatt jövőbeni adatokat talált a parser, ezért ezeket kézzel módosítottuk

In [ ]:
%matplotlib inline
import pandas
import matplotlib
import time
import random
import math
matplotlib.style.use("default")

In [ ]:
start = time.time()

# Load data (labels are not included)
raw_data = pandas.read_csv("data.csv", sep=";", encoding="UTF8", header=None, skiprows=1)

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", \
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", \
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", \
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", \
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

data, wrong_data, skipped_lines = [], [], 0
for i, d in raw_data.iterrows():
    try:
        # Transform or validate data
        date = pandas.to_datetime(d[0])
        posted = pandas.to_datetime(d[6])
        row = (date, d[1], d[2], d[3], d[4], d[5], posted, d[7])
        
        # Do some checks...
        if date.date() > posted.date():
            wrong_data.append(row)
            continue
        
        if state not in states:
            wrong_data.append(row)
            continue
        
        # Add data to cleaned array
        data.append(row)
    except Exception as e:
        skipped_lines += 1
        #print(d)

names = ["Date", "City", "State", "Shape", "Duration", "Summary", "Posted", "Description"]
data = pandas.DataFrame.from_records(data, columns=names)
wrong_data = pandas.DataFrame.from_records(wrong_data, columns=names)

print('Loading data took ' + str(time.time() - start) + 's')
print('Raw data: ' + str(len(raw_data)) + ' lines')
print('Processed data: ' + str(len(data)) + ' lines')
print('Skipped because of parser exception: ' + str(skipped_lines) + ' lines')
print('Wrong data: ' + str(len(wrong_data)) + ' lines')

In [ ]:
import json
import operator
from collections import Counter
from duration_parser import UFOParser

with open('data_old.json', 'r') as file:
    data = json.load(file)

In [ ]:
data.keys()

In [ ]:
cities = list(data['City'].values())

In [ ]:
cities

In [ ]:
import re

In [ ]:
def chk_word(word):
    if word is None:
        return False
    return re.match("^[a-zA-Z0-9-() ]*$", word) is not None

In [ ]:
bad_citites = [x for x in cities if chk_word(x) is False]

In [ ]:
len(bad_citites)

In [ ]:
bad_citites

In [ ]:
data['City']

In [ ]:
city = (data['City']['34139'], data['State']['34139'])
city

In [ ]:
import requests

In [ ]:
def download_geodata(name):
    url = 'http://nominatim.openstreetmap.org/search/'
    response = requests.get(url + name, params={'addressdetails': 1, 'format': 'json', 'limit': 10}).json()
    return [(d['address'], d['lon'], d['lat']) for d in response if d['type'] == 'city']

In [ ]:
geodata = download_geodata('Bedford')
geodata

In [ ]:
sts = {"Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", \
       "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", \
       "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", \
       "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", \
       "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD", \
       "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", \
       "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", \
       "NewHampshire": "NH", "NewJersey": "NJ", "NewMexico": "NM", "NewYork": "NY", \
       "NorthCarolina": "NC", "NorthDakota": "ND", "Ohio": "OH", "Oklahoma": "OK", \
       "Oregon": "OR", "Pennsylvania": "PA", "RhodeIsland": "RI", "SouthCarolina": "SC", \
       "SouthDakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", \
       "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "WestVirginia": "WV", \
       "Wisconsin": "WI", "Wyoming": "WY"}

In [ ]:
city

In [ ]:
def check_geodata(org_data, result):
    city, state = org_data
    if state.upper() in list(sts.values()):
        print('American')
        for item in result:
            print(item)
    
check_geodata(city, geodata)

## Adatok feltérképezése

### Használt oszlopok és típusai, valamint néhány mintaadat

In [ ]:
data.dtypes
sample_data = []
for i in range(10):
    sample_data.append(data.iloc[random.randint(0,len(data))])
pandas.DataFrame.from_records(sample_data, columns=names)

### Adatok vizualizációja az észlelés ideje alapján

In [ ]:
by_year = data.groupby(data["Date"].dt.year).count()["Date"]
print(by_year.describe())
data.groupby(data["Date"].dt.year).count()["Date"].plot(figsize=(15,5),kind="bar")

A vizualizációból látszik, hogy az adatok nagy része 2000 utánról származik.

### Adatok vizualizációja az észlelt ufók száma alapján, alak szerint csoportosítva

In [ ]:
shape_counts = data["Shape"].value_counts()
print(shape_counts.describe())
shape_counts.plot(figsize=(15,5),kind="bar")

### Adatok leíró statisztikái a City és a State oszlop szerint

In [ ]:
by_city = data.groupby("City").count()["Date"]
by_city.describe()

Az adatok város szerinti vizsgálatából látszik, hogy nagyon sok különböző várost adtak meg az adathalmazban, amiből sok olyan van, ami vagy egyszeri észlelés, vagy ami valószínűbb, hogy valami elírás, vagy hibásan felvitt adat.

In [ ]:
by_state = data.groupby("State").count()["Date"]
by_state.describe()

Az állam szerinti csoportosításnál úgy tűnik sikeres volt az adatok validációja és transzformációja, mert ezek szerint 1 kivételével mindegyik államból jelentettek észlelést. A qvartilisek, valamint a min és max értékek sem szembetűnően kiugróak a vártakhoz képest.

### Vizualizációs feladat megoldásának terve

A feladat szerint meg kell jelenítenünk az UFO észleléseket térbeli, és időbeli eloszlásuk szerint.
Erre megoldásnak az alábbi vizualizációt tervezzük megvalósítani:
* PySpark segítségével, Map - Reduce módszerrel elkészítjük az egyes évekhez tartozó megfigyelések számát, állam szerint
* Ezeket egy 2D pont diagrammon ábrázoljuk, X tengelyen az államokat, Y tengelyen az éveket.
* Ábrázolásnál az adatok számosságának jelölésére színeket és/vagy méreteket használunk.

A koncepciót az alábbi ábra szemlélteti:

In [ ]:
concept = [{"x":2,"y":1,"count":10},{"x":1,"y":2,"count":5},{"x":1,"y":5,"count":2},{"x":4,"y":1,"count":7}]
concept_df = pandas.DataFrame.from_records(concept)
matplotlib.pyplot.scatter(concept_df["x"],concept_df["y"],s=concept_df["count"]**2 * math.pi)


<SparkContext master=local[*] appName=PySparkShell>